# Infinispan

Infinispan 是一个开源键值数据网格，它可以作为单个节点运行，也可以分布式运行。

自 15.x 版本起支持向量搜索
更多信息请访问：[Infinispan Home](https://infinispan.org)

In [ ]:
# Ensure that all we need is installed
# You may want to skip this
%pip install sentence-transformers
%pip install langchain
%pip install langchain_core
%pip install langchain_community

# 设置

要运行此演示，我们需要一个正在运行的 Infinispan 实例，该实例不需要身份验证，并且需要一个数据文件。
在接下来的三个单元格中，我们将：
- 下载数据文件
- 创建配置
- 在 docker 中运行 Infinispan

In [ ]:
%%bash
#get an archive of news
wget https://raw.githubusercontent.com/rigazilla/infinispan-vector/main/bbc_news.csv.gz

In [ ]:
%%bash
#create infinispan configuration file
echo 'infinispan:
  cache-container: 
    name: default
    transport: 
      cluster: cluster 
      stack: tcp 
  server:
    interfaces:
      interface:
        name: public
        inet-address:
          value: 0.0.0.0 
    socket-bindings:
      default-interface: public
      port-offset: 0        
      socket-binding:
        name: default
        port: 11222
    endpoints:
      endpoint:
        socket-binding: default
        rest-connector:
' > infinispan-noauth.yaml

In [ ]:
!docker rm --force infinispanvs-demo
!docker run -d --name infinispanvs-demo -v $(pwd):/user-config  -p 11222:11222 infinispan/server:15.0 -c /user-config/infinispan-noauth.yaml

# 代码

## 选择一个嵌入模型

在此演示中，我们使用的是 HuggingFace 的嵌入模型。

In [ ]:
from langchain_core.embeddings import Embeddings
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L12-v2"
hf = HuggingFaceEmbeddings(model_name=model_name)

## 配置 Infinispan 缓存

Infinispan 是一个非常灵活的键值存储，它可以存储原始数据（raw bits）以及复杂的数据类型。
用户可以完全自由地配置数据网格，但对于简单的数据类型，所有配置都由 Python 层自动完成。我们利用了这一特性，以便我们可以专注于我们的应用程序。

## 准备数据

在此演示中，我们依赖于默认配置，因此文本、元数据和向量位于同一个缓存中。但也可以选择其他选项：例如，内容可以存储在其他地方，而向量存储可以仅包含对实际内容的引用。

In [ ]:
import csv
import gzip
import time

# Open the news file and process it as a csv
with gzip.open("bbc_news.csv.gz", "rt", newline="") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=",", quotechar='"')
    i = 0
    texts = []
    metas = []
    embeds = []
    for row in spamreader:
        # first and fifth values are joined to form the content
        # to be processed
        text = row[0] + "." + row[4]
        texts.append(text)
        # Store text and title as metadata
        meta = {"text": row[4], "title": row[0]}
        metas.append(meta)
        i = i + 1
        # Change this to change the number of news you want to load
        if i >= 5000:
            break

# 填充向量存储

In [ ]:
# add texts and fill vector db

from langchain_community.vectorstores import InfinispanVS

ispnvs = InfinispanVS.from_texts(texts, hf, metas)

# 用于打印结果文档的辅助函数

默认情况下，InfinispanVS 会将 protobuf 的 `text` 字段返回到 `Document.page_content` 中，并将所有剩余的 protobuf 字段（除了向量）返回到 `metadata` 中。此行为可以通过在设置时使用 lambda 函数进行配置。

In [ ]:
def print_docs(docs):
    for res, i in zip(docs, range(len(docs))):
        print("----" + str(i + 1) + "----")
        print("TITLE: " + res.metadata["title"])
        print(res.page_content)

# 试试看！！！

以下是一些示例查询

In [ ]:
docs = ispnvs.similarity_search("European nations", 5)
print_docs(docs)

In [ ]:
print_docs(ispnvs.similarity_search("Milan fashion week begins", 2))

In [ ]:
print_docs(ispnvs.similarity_search("Stock market is rising today", 4))

In [ ]:
print_docs(ispnvs.similarity_search("Why cats are so viral?", 2))

In [ ]:
print_docs(ispnvs.similarity_search("How to stay young", 5))

In [ ]:
!docker rm --force infinispanvs-demo